In [2]:
# Initial imports
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import pyEX as p

%matplotlib inline

In [3]:
# Load .env environment variables
load_dotenv('')

True

In [4]:
# Set IEX API key and secret
from pathlib import Path
env_file = Path('API_keys.env')
load_dotenv(env_file)

True

In [5]:
# Set variable for IEX API key
iex_secret_key = os.getenv("IEX_SECRET_KEY")

In [6]:
# Verify API Key
type(iex_secret_key)

str

In [7]:
# View chart endpoint details
p.chart?

Signature:
p.chart(
    symbol,
    timeframe='1m',
    date=None,
    exactDate=None,
    last=-1,
    closeOnly=False,
    byDay=False,
    simplify=False,
    interval=-1,
    changeFromClose=False,
    displayPercent=False,
    sort='desc',
    includeToday=False,
    token='',
    version='stable',
    filter='',
    format='json',
)
Docstring:
Historical price/volume data, daily and intraday

https://iexcloud.io/docs/api/#historical-prices
Data Schedule
1d: -9:30-4pm ET Mon-Fri on regular market trading days
    -9:30-1pm ET on early close trading days
All others:
    -Prior trading day available after 4am ET Tue-Sat

Args:
    symbol (str): Ticker to request
    timeframe (str): Timeframe to request e.g. 1m
    date (datetime): date, if requesting intraday
    exactDate (str): Same as `date`, takes precedence
    last (int): If passed, chart data will return the last N elements from the time period defined by the range parameter
    closeOnly (bool): Will return adjusted data on

In [8]:
# View financials endpoint details
p.financials?

Signature:
p.financials(
    symbol,
    period='quarter',
    token='',
    version='stable',
    filter='',
    format='json',
)
Docstring:
Pulls income statement, balance sheet, and cash flow data from the four most recent reported quarters.

https://iexcloud.io/docs/api/#financials
Updates at 8am, 9am UTC daily

Args:
    symbol (str): Ticker to request
    period (str): Period, either 'annual' or 'quarter'
    token (str): Access token
    version (str): API version
    filter (str): filters: https://iexcloud.io/docs/api/#filter-results
    format (str): return format, defaults to json

Returns:
    dict or DataFrame: result
File:      c:\users\mdetw\anaconda3\lib\site-packages\pyex\stocks\fundamentals.py
Type:      function


In [9]:
# Import Pathlib
from pathlib import Path 

In [12]:
# Pull in tech stocks csv
tech_df = pd.read_csv(Path('tech stocks.csv'))        
tech_df.head(5)

stocks
0   AAPL
1   MSFT
2   AMZN
3     FB
4   NVDA

In [13]:
# Read consumer staples stocks 
staples_df = pd.read_csv(Path('consumer staples stocks.csv'))   

## Pull tech stock data

### Pull tech stock 'financial' endpoint

In [286]:
# Call Tech stock data from IEX API
tech_stock_data_list = []

for ticker in tech_df['stocks']:
    tech_stock_data = p.financials(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, EBITDA, totalDebt, totalAssets, shareholderEquity, netIncome, cashFlow, totalRevenue')
    tech_stock_df = pd.DataFrame(tech_stock_data, index=[0])
    tech_stock_data_list.append(tech_stock_df)
    
tech_stock_df_final = pd.concat(tech_stock_data_list)
tech_stock_df_final = tech_stock_df_final.reset_index()
tech_stock_df_final = tech_stock_df_final.drop(columns=["index"])
tech_stock_df_final.head()

symbol       EBITDA     totalDebt   totalAssets  shareholderEquity  \
0   AAPL  53550000000  249438000000  702004000000       126180000000   
1   MSFT  23450000000   53288000000  335418000000       151978000000   
2   AMZN  13796000000   65862000000  382406000000       120564000000   
3     FB  12418000000             0  169585000000       133360000000   
4   NVDA   2969000000   10944000000   40632000000        23798000000   

     netIncome     cashFlow  totalRevenue  
0  41102000000  40400000000  166720000000  
1  20505000000  24540000000   45317000000  
2   3156000000   7313000000  110812000000  
3   9194000000  14090000000   29010000000  
4   2464000000   1519000000    7103000000

### Pull tech stock 'quote' endpoint

In [256]:
tech_quote_data_list = []

for ticker in tech_df['stocks']:
    tech_quote_data = p.quote(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, peRatio, netIncome, marketCap, cashFlow, totalRevenue')
    tech_quote_df = pd.DataFrame(tech_quote_data, index=[0])
    tech_quote_data_list.append(tech_quote_df)
    
tech_quote_df_final = pd.concat(tech_quote_data_list)
tech_quote_df_final = tech_quote_df_final.reset_index()
tech_quote_df_final = tech_quote_df_final.drop(columns=["index"])
tech_quote_df_final.head()

symbol  peRatio      marketCap
0   AAPL    14.66  2688800810310
1   MSFT    33.73  2263881343279
2   AMZN    59.33  1538041801603
3     FB    22.65   903776220534
4   NVDA    74.54   603800000000

## Pull consumer staples stock data

### Pull consumer staples stock 'financial' endpoint

In [261]:
staples_fin_list = []

for ticker in staples_df['Stocks']:
    staples_fin_data = p.financials(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, EBITDA, totalDebt, totalAssets, shareholderEquity, netIncome, cashFlow, totalRevenue')
    staples_fin_df = pd.DataFrame(staples_fin_data)
    staples_fin_list.append(staples_fin_df)
        
staples_fin_df_final = pd.concat(staples_fin_list)
staples_fin_df_final = staples_fin_df_final.reset_index()
staples_fin_df_final = staples_fin_df_final.drop(columns=["index"])
staples_fin_df_final.head()

symbol        EBITDA     totalDebt   totalAssets  shareholderEquity  \
0     PG  5.734000e+09  3.254700e+10  1.196690e+11       4.611100e+10   
1    PEP  3.809000e+09  4.125700e+10  9.325400e+10       1.587200e+10   
2     KO  3.760000e+09  4.170800e+10  9.060600e+10       2.217900e+10   
3     PM  3.522000e+09  2.910500e+10  4.158900e+10      -1.055100e+10   
4    CVS  4.626000e+09  5.839300e+10  2.348910e+11       7.430800e+10   

      netIncome      cashFlow  totalRevenue  
0  4.042000e+09  4.643000e+09  2.033800e+10  
1  2.224000e+09  4.294000e+09  2.018900e+10  
2  2.471000e+09  3.706000e+09  1.004200e+10  
3  2.419000e+09  3.870000e+09  2.161700e+10  
4  1.598000e+09  5.521000e+09  7.379400e+10

### Pull consumer staple stock 'quote' endpoint

In [262]:
staples_quote_data_list = []

for ticker in staples_df['Stocks']:
    staples_quote_data = p.quote(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, peRatio, marketCap')
    staples_quote_df = pd.DataFrame(staples_quote_data, index=[0])
    staples_quote_data_list.append(staples_quote_df)
    
staples_quote_df_final = pd.concat(staples_quote_data_list)
staples_quote_df_final = staples_quote_df_final.reset_index()
staples_quote_df_final = staples_quote_df_final.drop(columns=["index"])
staples_quote_df_final.head()

symbol  peRatio     marketCap
0     PG    30.17  392031546678
1    PEP    29.68  240498652905
2     KO    29.93  262404779337
3     PM    17.74  158827559914
4    CVS    17.87  135160805890

In [14]:
# Concat consumer staples dataframes
staples_final_df = pd.concat([staples_fin_df_final, staples_quote_df_final], axis="columns", join="inner")
staples_final_df.head()

NameError: name 'staples_fin_df_final' is not defined

symbol        EBITDA     totalDebt   totalAssets  shareholderEquity  \
0     PG  5.734000e+09  3.254700e+10  1.196690e+11       4.611100e+10   
1    PEP  3.809000e+09  4.125700e+10  9.325400e+10       1.587200e+10   
2     KO  3.760000e+09  4.170800e+10  9.060600e+10       2.217900e+10   
3     PM  3.522000e+09  2.910500e+10  4.158900e+10      -1.055100e+10   
4    CVS  4.626000e+09  5.839300e+10  2.348910e+11       7.430800e+10   

      netIncome      cashFlow  totalRevenue symbol  peRatio     marketCap  
0  4.042000e+09  4.643000e+09  2.033800e+10     PG    30.17  392031546678  
1  2.224000e+09  4.294000e+09  2.018900e+10    PEP    29.68  240498652905  
2  2.471000e+09  3.706000e+09  1.004200e+10     KO    29.93  262404779337  
3  2.419000e+09  3.870000e+09  2.161700e+10     PM    17.74  158827559914  
4  1.598000e+09  5.521000e+09  7.379400e+10    CVS    17.87  135160805890

In [272]:
staples_fin_df_final.set_index('symbol')

EBITDA     totalDebt   totalAssets  shareholderEquity  \
symbol                                                                
PG      5.734000e+09  3.254700e+10  1.196690e+11       4.611100e+10   
PEP     3.809000e+09  4.125700e+10  9.325400e+10       1.587200e+10   
KO      3.760000e+09  4.170800e+10  9.060600e+10       2.217900e+10   
PM      3.522000e+09  2.910500e+10  4.158900e+10      -1.055100e+10   
CVS     4.626000e+09  5.839300e+10  2.348910e+11       7.430800e+10   
MO     -2.902000e+09  2.812700e+10  3.956400e+10      -1.267000e+09   
MDLZ    1.743000e+09  1.998700e+10  6.731700e+10       2.769800e+10   
CL      1.127000e+09  7.696000e+09  1.588000e+10       5.910000e+08   
KMB     8.790000e+08  8.942000e+09  1.777500e+10       4.740000e+08   
GIS     9.765000e+08  1.267230e+10  3.248160e+10       9.524500e+09   
SYY     8.463690e+08  1.114558e+10  2.210287e+10       1.673292e+09   
ADM     9.630000e+08  8.934000e+09  5.229800e+10       2.196900e+10   
MCK     9.400000e+08  5.985000e+09  6.360100e+10      -5.710000e+08   
WBA     1.777000e+09  1.384600e+10  9.650600e+10       2.626300e+10   
STZ     9.262000e+08  1.033250e+10  2.552730e+10       1.123830e+10   
KR      1.527000e+09  1.372100e+10  4.982900e+10       9.519000e+09   
CTVA   -1.570000e+08  2.473000e+09  4.112200e+10       2.474000e+10   
MNST    4.576910e+08  0.000000e+00  7.449408e+09       6.244997e+09   
HSY     6.532200e+08  4.508345e+09  9.439183e+09       2.528294e+09   
KDP     9.650000e+08  1.333600e+10  5.033000e+10       2.440500e+10   
TSN     4.434000e+09  1.869600e+10  7.261800e+10       3.544600e+10   
KHC     1.384000e+09  2.397200e+10  9.489700e+10       5.008900e+10   
CHD     3.580000e+08  1.794100e+09  7.382000e+09       3.513600e+09   
MKC     3.286000e+08  5.520900e+09  1.288440e+10       4.386500e+09   
CLX     2.740000e+08  2.870000e+09  6.274000e+09       3.680000e+08   
ABC     1.707446e+09  1.336785e+10  1.146756e+11       4.467080e+08   
K       5.660000e+08  7.533000e+09  1.840700e+10       3.484000e+09   
CAG     5.359000e+08  9.384200e+09  2.259200e+10       8.679000e+09   
SJM     3.685000e+08  4.628800e+09  1.624080e+10       8.286400e+09   
HRL     8.471940e+08  6.647806e+09  2.539266e+10       1.394577e+10   
BG      7.000000e+08  6.475000e+09  2.410900e+10       7.495000e+09   
IFF     4.980000e+08  1.153000e+10  4.025200e+10       2.142600e+10   
DAR     2.846350e+08  1.380874e+09  5.925720e+09       3.205524e+09   
LW      1.721000e+08  2.724300e+09  4.155600e+09       3.652000e+08   
TAP     7.291000e+08  7.220800e+09  2.764350e+10       1.327550e+10   
CPB     4.580000e+08  5.047000e+09  1.189900e+10       3.235000e+09   
USFD    2.330000e+08  5.578000e+09  1.311700e+10       4.182000e+09   
CASY    2.169460e+08  1.711525e+09  5.252110e+09       2.123134e+09   
POST    3.240000e+08  1.408340e+10  2.482940e+10       5.484800e+09   
INGR    2.110000e+08  2.146000e+09  6.986000e+09       3.084000e+09   
FLO     8.614700e+07  9.077130e+08  3.323396e+09       1.411930e+09   
ACI     1.121400e+09  7.997700e+09  2.793610e+10       3.909800e+09   
HLF     2.160000e+08  2.762700e+09  2.853000e+09      -1.333400e+09   
SAM    -7.408000e+06  0.000000e+00  1.463526e+09       1.029378e+09   
SPB    -4.108000e+08  5.012600e+09  1.068080e+10       2.943800e+09   
BYND   -4.236900e+07  1.129361e+09  1.432374e+09       2.071050e+08   
FRPT    6.367000e+06  0.000000e+00  7.583270e+08       7.235180e+08   
HAIN    2.692000e+07  3.457490e+08  2.142265e+09       1.348756e+09   
GO      4.363800e+07  4.508600e+08  2.617668e+09       9.940310e+08   
REYN    1.330000e+08  2.117000e+09  4.733000e+09       1.694000e+09   
PPC     2.128530e+08  3.280659e+09  8.832843e+09       2.604243e+09   
SEB     1.550000e+08  1.184000e+09  7.257000e+09       4.273000e+09   
EL      1.124000e+09  5.548000e+09  2.158000e+10       6.081000e+09   
ABEV    9.611774e+08  5.981498e+08  2.495212e+10       1.602218e+10   

           netIncome      cashFlow  t

In [273]:
staples_quote_df_final.set_index('symbol')

peRatio     marketCap
symbol                       
PG        30.17  392031546678
PEP       29.68  240498652905
KO        29.93  262404779337
PM        17.74  158827559914
CVS       17.87  135160805890
MO        33.78   92455647411
MDLZ      21.71   94509361943
CL        26.61   70200867473
KMB       24.08   47668986334
GIS       18.87   41307594816
SYY       58.67   40002574444
ADM       14.88   38539908401
MCK       -8.25   38451476685
WBA        7.17   45908806396
STZ     -895.18   45737985256
KR        35.54   34755530510
CTVA      21.15   33923424860
MNST      30.31   45712328145
HSY       28.95   41453560720
KDP       31.49   54024336731
TSN        5.42   32939742560
KHC       19.82   45375254029
CHD       31.19   24898250365
MKC       33.93   25430134303
CLX       50.86   21621389561
ABC        9.00   27656761010
K         18.02   22558450962
CAG       15.55   17005284564
SJM       20.13   15294384306
HRL       14.73   26542521905
BG         6.13   13315694057
IFF      120.51   35891126154
DAR       20.08   10460900970
LW        49.49    9486151188
TAP      -24.20   10714228425
CPB       14.32   13526909071
USFD     202.63    7895700352
CASY      23.29    6846203065
POST      22.34    7143518585
INGR      40.86    6634297780
FLO       27.00    5993042041
ACI       18.55   13172922180
HLF       10.65    5075136864
SAM       56.72    4629994611
SPB       10.49    3926535913
BYND     -30.01    3891413641
FRPT    -155.54    3843528896
HAIN      37.65    3419119598
FMX         NaN   17609276470
GO        31.23    2400426075
OLPX        NaN   13150448986
REYN      19.69    6524599482
PPC    -1182.83    6715697386
SEB        6.14    4381151395
CCEP        NaN   26530067111
UL        18.93  126914855324
DEO       16.26  118905091278
EL        36.49  108865405828
BTI       11.20   97636405869
ABEV      13.82   41716889763

In [297]:
# concat consumer staples dataframes
staples_final_df = pd.concat([staples_fin_df_final, staples_quote_df_final], axis="columns", join="inner")

In [306]:
staples_final_df['sector'] = 'staples'
staples_final_df = staples_final_df.loc[:,~staples_final_df.columns.duplicated()]
staples_final_df

symbol        EBITDA     totalDebt   totalAssets  shareholderEquity  \
0      PG  5.734000e+09  3.254700e+10  1.196690e+11       4.611100e+10   
1     PEP  3.809000e+09  4.125700e+10  9.325400e+10       1.587200e+10   
2      KO  3.760000e+09  4.170800e+10  9.060600e+10       2.217900e+10   
3      PM  3.522000e+09  2.910500e+10  4.158900e+10      -1.055100e+10   
4     CVS  4.626000e+09  5.839300e+10  2.348910e+11       7.430800e+10   
5      MO -2.902000e+09  2.812700e+10  3.956400e+10      -1.267000e+09   
6    MDLZ  1.743000e+09  1.998700e+10  6.731700e+10       2.769800e+10   
7      CL  1.127000e+09  7.696000e+09  1.588000e+10       5.910000e+08   
8     KMB  8.790000e+08  8.942000e+09  1.777500e+10       4.740000e+08   
9     GIS  9.765000e+08  1.267230e+10  3.248160e+10       9.524500e+09   
10    SYY  8.463690e+08  1.114558e+10  2.210287e+10       1.673292e+09   
11    ADM  9.630000e+08  8.934000e+09  5.229800e+10       2.196900e+10   
12    MCK  9.400000e+08  5.985000e+09  6.360100e+10      -5.710000e+08   
13    WBA  1.777000e+09  1.384600e+10  9.650600e+10       2.626300e+10   
14    STZ  9.262000e+08  1.033250e+10  2.552730e+10       1.123830e+10   
15     KR  1.527000e+09  1.372100e+10  4.982900e+10       9.519000e+09   
16   CTVA -1.570000e+08  2.473000e+09  4.112200e+10       2.474000e+10   
17   MNST  4.576910e+08  0.000000e+00  7.449408e+09       6.244997e+09   
18    HSY  6.532200e+08  4.508345e+09  9.439183e+09       2.528294e+09   
19    KDP  9.650000e+08  1.333600e+10  5.033000e+10       2.440500e+10   
20    TSN  4.434000e+09  1.869600e+10  7.261800e+10       3.544600e+10   
21    KHC  1.384000e+09  2.397200e+10  9.489700e+10       5.008900e+10   
22    CHD  3.580000e+08  1.794100e+09  7.382000e+09       3.513600e+09   
23    MKC  3.286000e+08  5.520900e+09  1.288440e+10       4.386500e+09   
24    CLX  2.740000e+08  2.870000e+09  6.274000e+09       3.680000e+08   
25    ABC  1.707446e+09  1.336785e+10  1.146756e+11       4.467080e+08   
26      K  5.660000e+08  7.533000e+09  1.840700e+10       3.484000e+09   
27    CAG  5.359000e+08  9.384200e+09  2.259200e+10       8.679000e+09   
28    SJM  3.685000e+08  4.628800e+09  1.624080e+10       8.286400e+09   
29    HRL  8.471940e+08  6.647806e+09  2.539266e+10       1.394577e+10   
30     BG  7.000000e+08  6.475000e+09  2.410900e+10       7.495000e+09   
31    IFF  4.980000e+08  1.153000e+10  4.025200e+10       2.142600e+10   
32    DAR  2.846350e+08  1.380874e+09  5.925720e+09       3.205524e+09   
33     LW  1.721000e+08  2.724300e+09  4.155600e+09       3.652000e+08   
34    TAP  7.291000e+08  7.220800e+09  2.764350e+10       1.327550e+10   
35    CPB  4.580000e+08  5.047000e+09  1.189900e+10       3.235000e+09   
36   USFD  2.330000e+08  5.578000e+09  1.311700e+10       4.182000e+09   
37   CASY  2.169460e+08  1.711525e+09  5.252110e+09       2.123134e+09   
38   POST  3.240000e+08  1.408340e+10  2.482940e+10       5.484800e+09   
39   INGR  2.110000e+08  2.146000e+09  6.986000e+09       3.084000e+09   
40    FLO  8.614700e+07  9.077130e+08  3.323396e+09       1.411930e+09   
41    ACI  1.121400e+09  7.997700e+09  2.793610e+10       3.909800e+09   
42    HLF  2.160000e+08  2.762700e+09  2.853000e+09      -1.333400e+09   
43    SAM -7.408000e+06  0.000000e+00  1.463526e+09       1.029378e+09   
44    SPB -4.108000e+08  5.012600e+09  1.068080e+10       2.943800e+09   
45   BYND -4.236900e+07  1.129361e+09  1.432374e+09       2.071050e+08   
46   FRPT  6.367000e+06  0.000000e+00  7.583270e+08       7.235180e+08   
47   HAIN  2.692000e+07  3.457490e+08  2.142265e+09       1.348756e+09   
48     GO  4.363800e+07  4.508600e+08  2.617668e+09       9.940310e+08   
49   REYN  1.330000e+08  2.117000e+09  4.733000e+09       1.694000e+09   
50    PPC  2.128530e+08  3.280659e+09  8.832843e+09       2.604243e+09   
51    SEB  1.550000e+08  1.184000e+09  7.257000e+09       4.273000e+09   
52     EL  1.124000e+09  5.548000e+09  2.158000e+10       6.081000e+09   
53   AB

In [288]:
# concat tech stocks dataframes
tech_final_df = pd.concat([tech_stock_df_final, tech_quote_df_final], axis="columns", join="inner")

In [311]:
# Drop duplicate columns
tech_final_df = tech_final_df.loc[:,~tech_final_df.columns.duplicated()]
tech_final_df["sector"] = 'technology'
tech_final_df

symbol       EBITDA     totalDebt   totalAssets  shareholderEquity  \
0    AAPL  53550000000  249438000000  702004000000       126180000000   
1    MSFT  23450000000   53288000000  335418000000       151978000000   
2    AMZN  13796000000   65862000000  382406000000       120564000000   
3      FB  12418000000             0  169585000000       133360000000   
4    NVDA   2969000000   10944000000   40632000000        23798000000   
5   GOOGL  24523000000   14288000000  347403000000       244567000000   
6    KEYS    778000000    3582000000   15562000000         7568000000   
7       V   9036000000   41954000000  165792000000        75178000000   
8      MA   2932000000   13861000000   35410000000         6703000000   
9    CSCO   3527000000    9502000000   95981000000        42701000000   
10   ADBE   1484000000    4122000000   26144000000        14414000000   
11   AVGO   7900000000   79460000000  151140000000        49978000000   
12   NFLX   4788557000   15493164000   42739857000        15314617000   
13   INTC   8228000000   40304000000  167962000000        90087000000   
14    CRM   -403000000   10591000000   87436000000        57054000000   
15    ACN   2935159000      64973000   43674743000        20353199000   
16   QCOM   6658000000   31490000000   82480000000        19900000000   
17   PYPL   1367000000    7949000000   74534000000        22090000000   
18    TXN   2605000000    7739000000   23273000000        12151000000   
19    AMD   1059000000     313000000   11153000000         7136000000   
20   INTU    263000000    2037000000   14870000000         9733000000   
21   AMAT   4236000000   10904000000   51650000000        24494000000   
22   ORCL   4174000000   78431000000  106897000000       -10101000000   
23   SHOP      9503000     910376000   13536518000        11370277000   
24    IBM   3526000000   54501000000  144215000000        22228000000   
25     MU   4344000000    7022000000   61246000000        45908000000   
26    NOW    199000000    1577000000    9478000000         3534000000   
27   LRCX   1452107000    4995332000   15565552000         5820043000   
28    ADP    993500000    2985500000   59987200000         5350700000   
29    ADI    810488000   13539750000  104644142000        75985084000   
30    FIS   1397000000   19780000000   81690000000        47105000000   
31   MRVL    323394000    4556526000   22018842000        15686344000   
32   KLAC    903624000    3443436000   11146477000         3860930000   
33   SNOW   -151753000             0    6163471000         5006682000   
34   FISV   1463000000   20989000000   75767000000        31596000000   
35   NXPI    851000000    9593000000   20010000000         6738000000   
36   ADSK    219000000    2626800000    8531000000         1279900000   
37     SQ    129414000    4742466000   13909426000         2959825000   
38    TEL   1770000000    8184000000   42924000000        21268000000   
39   ATVI    923000000    3607000000   23977000000        16950000000   
40   SNPS    434376000     200172000   17504520000        10590274000   
41   XLNX    262243000    1493311000    5995561000         3395545000   
42   PANW     -1000000    3672100000   10426400000          518900000   
43    APH    666400000    5248700000   14926100000         5945900000   
44   WDAY    -56774000    1848209000    9440883000         4215816000   
45   MCHP    642200000    8200600000   16215000000         5702900000   
46   CTSH    873000000     674000000   17232000000        11546000000   
47   CDNS    198812000     347385000    4213875000         2631262000   
48    HPQ   2692000000   14984000000   77220000000        -3300000000   
49    GPN    850009000   11337286000   45754475000        26147027000   
50    MSI    511000000    5693000000   11422000000         -263000000   
51   FTNT    185400000    1005600000    5970600000         1118100000   
52   SNAP   -176584000    2252079000    7111165000         3440116000   
53   PAYX    488900000     806400000    9688100000         31

In [312]:
# Open utility stocks csv and save to df
utilities_df = pd.read_csv(Path('utility stocks.csv'))   

In [320]:
# API call for utility stocks financials
utilities_fin_list = []

for ticker in utilities_df['STOCKS']:
    utilities_fin_data = p.financials(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, EBITDA, totalDebt, totalAssets, shareholderEquity, netIncome, cashFlow, totalRevenue')
    utilities_fin_df = pd.DataFrame(utilities_fin_data)
    utilities_fin_list.append(utilities_fin_df)
        
utilities_fin_df_final = pd.concat(utilities_fin_list)
utilities_fin_df_final = utilities_fin_df_final.reset_index()
utilities_fin_df_final = utilities_fin_df_final.drop(columns=["index"])
utilities_fin_df_final.head()

symbol       EBITDA     totalDebt   totalAssets  shareholderEquity  \
0    VST  587000000.0  1.087500e+10  2.993200e+10       6.036000e+09   
1    WEC  703900000.0  1.468370e+10  3.840090e+10       1.093890e+10   
2     SR  168800000.0  7.333800e+09  1.871280e+10       5.316400e+09   
3    AGR  396000000.0  7.920000e+09  3.887900e+10       1.910200e+10   
4     ES  805413000.0  1.942709e+10  4.801406e+10       1.441161e+10   

     netIncome   cashFlow  totalRevenue  
0    7000000.0  564000000  2.991000e+09  
1  290000000.0  780500000  1.746500e+09  
2  -27000000.0   58200000  5.804000e+08  
3  111000000.0  143000000  1.598000e+09  
4  283166000.0  712828000  2.432794e+09

In [319]:
# API call for utility stocks quote
utilities_quote_data_list = []

for ticker in utilities_df['STOCKS']:
    utilities_quote_data = p.quote(ticker, token = iex_secret_key, version = 'stable', filter = 'symbol, peRatio, marketCap')
    utilities_quote_df = pd.DataFrame(utilities_quote_data, index=[0])
    utilities_quote_data_list.append(utilities_quote_df)
    
utilities_quote_df_final = pd.concat(utilities_quote_data_list)
utilities_quote_df_final = utilities_quote_df_final.reset_index()
utilities_quote_df_final = utilities_quote_df_final.drop(columns=["index"])
utilities_quote_df_final.head()

symbol  peRatio    marketCap
0    VST    -5.22  10487496010
1    WEC    23.24  30493056112
2     SR     6.50   3336320227
3    AGR    22.12  17900466624
4     ES    25.19  29797649726

In [321]:
# concat tech stocks dataframes
utilities_final_df = pd.concat([utilities_fin_df_final, utilities_quote_df_final], axis="columns", join="inner")

In [322]:
# Drop duplicate columns
utilities_final_df = utilities_final_df.loc[:,~utilities_final_df.columns.duplicated()]
utilities_final_df["sector"] = 'utilities'
utilities_final_df

symbol        EBITDA     totalDebt   totalAssets  shareholderEquity  \
0     VST  5.870000e+08  1.087500e+10  2.993200e+10       6.036000e+09   
1     WEC  7.039000e+08  1.468370e+10  3.840090e+10       1.093890e+10   
2      SR  1.688000e+08  7.333800e+09  1.871280e+10       5.316400e+09   
3     AGR  3.960000e+08  7.920000e+09  3.887900e+10       1.910200e+10   
4      ES  8.054130e+08  1.942709e+10  4.801406e+10       1.441161e+10   
5    WTRG  1.759590e+08  5.729463e+09  1.423648e+10       5.127862e+09   
6     OGS  9.295700e+07  4.019137e+09  8.133918e+09       2.315098e+09   
7    CWEN  2.410000e+08  7.844000e+09  1.178000e+10       1.777000e+09   
8     NWN  1.404700e+07  1.315804e+09  3.929225e+09       8.914610e+08   
9    EVRG  8.277000e+08  1.078420e+10  2.818210e+10       9.317800e+09   
10    DUK  3.176000e+09  6.490000e+10  1.670070e+11       4.930700e+10   
11    AWK  5.780000e+08  1.108100e+10  2.533300e+10       6.869000e+09   
12    POR  1.840000e+08  3.426000e+09  9.542000e+09       2.675000e+09   
13    NEW -1.847855e+07  2.333341e+08  5.145775e+08      -1.329353e+08   
14    ORA  8.521700e+07  1.933107e+09  4.359366e+09       1.833385e+09   
15    ALE  9.300000e+07  2.026800e+09  6.332200e+09       2.340000e+09   
16    LNT  4.670000e+08  7.391000e+09  1.832900e+10       5.985000e+09   
17    AVA  8.196600e+07  2.445358e+09  6.731478e+09       2.101636e+09   
18    UTL  3.680000e+07  5.419000e+08  1.509000e+09       4.396000e+08   
19    UGI  1.824000e+09  1.363200e+10  3.344600e+10       1.104400e+10   
20    SWX  1.146800e+08  4.143054e+09  1.036964e+10       2.909816e+09   
21     SO  2.788000e+09  5.283600e+10  1.278610e+11       2.868900e+10   
22    EIX  5.490000e+08  2.716000e+10  7.302300e+10       1.484100e+10   
23    SJI  2.476700e+07  3.404445e+09  7.024011e+09       1.828587e+09   
24    SJW  5.761100e+07  1.568392e+09  3.469741e+09       1.000711e+09   
25    PNM  2.236030e+08  3.514133e+09  8.256070e+09       2.156479e+09   
26    PEG  2.390000e+08  1.978000e+10  4.942600e+10       1.406900e+10   
27    PCG  1.199000e+09  4.294700e+10  1.035610e+11       2.047500e+10   
28    PPL  7.050000e+08  1.113900e+10  3.417100e+10       1.457600e+10   
29    EXC  3.414000e+09  4.170100e+10  1.326210e+11       3.385100e+10   
30   OTTR  9.734300e+07  8.624380e+08  2.713148e+09       9.524820e+08   
31    OGE  4.346000e+08  4.879000e+09  1.217750e+10       3.814900e+09   
32     NI  3.349000e+08  9.623900e+09  2.277860e+10       6.494900e+09   
33    NJR  6.614000e+07  5.224608e+09  1.144456e+10       3.261724e+09   
34    NFG  4.318880e+08  5.574374e+09  1.492965e+10       3.572412e+09   
35    XEL  1.375000e+09  2.334700e+10  5.751700e+10       1.517100e+10   
36   MSEX  1.692800e+07  3.232890e+08  1.029592e+09       3.644240e+08   
37   MGEE  5.658900e+07  6.379980e+08  2.370734e+09       1.028285e+09   
38    MDU  2.643980e+08  2.378266e+09  8.568863e+09       3.332278e+09   
39    DUK  3.176000e+09  6.490000e+10  1.670070e+11       4.930700e+10   
40    CNP  6.310000e+08  1.748500e+10  3.716000e+10       8.866000e+09   
41   YORW  8.907000e+06  1.321970e+08  4.307480e+08       1.506600e+08   
42     HE  1.653990e+08  2.466346e+09  1.576016e+10       2.417836e+09   
43    NEE  1.705000e+09  5.534100e+10  1.391630e+11       3.664900e+10   
44    ETR  1.249476e+09  2.584274e+10  5.884350e+10       1.125252e+10   
45    ATO  4.313440e+08  1.466131e+10  3.921732e+10       1.581378e+10   
46    IDA  1.723660e+08  2.000584e+09  7.254054e+09       2.670609e+09   
47    DTE  7.510000e+08  1.760900e+10  3.943500e+10       8.567000e+09   

       netIncome    cashFlow  totalRevenue  peRatio     marketCap     sector  
0   7.000000e+06   564000000  2.991000e+09    -5.22   10487496010  utilities  
1   2.900000e+08   780500000  1.746500e+09    23.24   30493056112  utilities  
2  -2.700000e+07    58200000  5.804000e+08     6.50    3336320227  utilities  
3   1.110000e+08   143000000  1.598000e+09    22.12   179004

In [336]:
# concat all sector dataframes and set symbol column as index
all_stocks_df = pd.concat([utilities_final_df, tech_final_df, staples_final_df], axis=0, join="inner")
all_stocks_df.set_index('symbol')

EBITDA     totalDebt   totalAssets  shareholderEquity  \
symbol                                                                
VST     5.870000e+08  1.087500e+10  2.993200e+10       6.036000e+09   
WEC     7.039000e+08  1.468370e+10  3.840090e+10       1.093890e+10   
SR      1.688000e+08  7.333800e+09  1.871280e+10       5.316400e+09   
AGR     3.960000e+08  7.920000e+09  3.887900e+10       1.910200e+10   
ES      8.054130e+08  1.942709e+10  4.801406e+10       1.441161e+10   
...              ...           ...           ...                ...   
REYN    1.330000e+08  2.117000e+09  4.733000e+09       1.694000e+09   
PPC     2.128530e+08  3.280659e+09  8.832843e+09       2.604243e+09   
SEB     1.550000e+08  1.184000e+09  7.257000e+09       4.273000e+09   
EL      1.124000e+09  5.548000e+09  2.158000e+10       6.081000e+09   
ABEV    9.611774e+08  5.981498e+08  2.495212e+10       1.602218e+10   

           netIncome      cashFlow  totalRevenue  peRatio    marketCap  \
symbol                                                                   
VST     7.000000e+06  5.640000e+08  2.991000e+09    -5.22  10487496010   
WEC     2.900000e+08  7.805000e+08  1.746500e+09    23.24  30493056112   
SR     -2.700000e+07  5.820000e+07  5.804000e+08     6.50   3336320227   
AGR     1.110000e+08  1.430000e+08  1.598000e+09    22.12  17900466624   
ES      2.831660e+08  7.128280e+08  2.432794e+09    25.19  29797649726   
...              ...           ...           ...      ...          ...   
REYN    6.600000e+07  1.040000e+08  9.050000e+08    31.23   2400426075   
PPC     6.072500e+07  2.125430e+08  3.827566e+09      NaN  13150448986   
SEB     9.400000e+07  1.640000e+08  2.276000e+09    19.69   6524599482   
EL      6.920000e+08 -8.100000e+07  4.392000e+09 -1182.83   6715697386   
ABEV    6.559632e+08  1.101450e+09  3.414628e+09     6.14   4381151395   

           sector  
symbol             
VST     utilities  
WEC     utilities  
SR      utilities  
AGR     utilities  
ES      utilities  
...           ...  
REYN      staples  
PPC       staples  
SEB       staples  
EL        staples  
ABEV      staples  

[162 rows x 10 columns]

In [347]:
# export all_stocks dataframe to csv file
stock_data_csv = all_stocks_df.to_csv('all_stock_data.csv', index = True)